In [27]:
import os
import sys
import tensorflow as tf

# we need to append the root directory to the path to import data.py\n",
sys.path.append('..')
import utils
import model

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])

In [24]:
class MobilNet_Architecture(tf.keras.models.Model):
    def __init__(self,width_multiplier, **kwargs):
        super(MobilNet_Architecture, self).__init__(**kwargs)       
        
        self.width_multiplier = width_multiplier
        
    def build(self, input_shape):
        
        input_channels = int(input_shape[-1])

        self.conv1 = tf.keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.activation1 = tf.keras.layers.Activation('relu')
        
        self.depthwise_block1 = DepthwiseSeparableConvolution(conv_filters=(32,64),conv_strides=(1,1),width_multiplier=self.width_multiplier)
        self.depthwise_block2 = DepthwiseSeparableConvolution(conv_filters=(64,128),conv_strides=(2,1),width_multiplier=self.width_multiplier)
        self.depthwise_block3 = DepthwiseSeparableConvolution(conv_filters=(128,128),conv_strides=(1,1),width_multiplier=self.width_multiplier)
        self.depthwise_block4 = DepthwiseSeparableConvolution(conv_filters=(128,256),conv_strides=(2,1),width_multiplier=self.width_multiplier)
        self.depthwise_block5 = DepthwiseSeparableConvolution(conv_filters=(256,256),conv_strides=(1,1),width_multiplier=self.width_multiplier)
        self.depthwise_block6 = DepthwiseSeparableConvolution(conv_filters=(256,512),conv_strides=(2,1),width_multiplier=self.width_multiplier)
        
        self.dw_penta_block = [
            DepthwiseSeparableConvolution(conv_filters=(512,512),conv_strides=(1,1),width_multiplier=self.width_multiplier)
            for _ in range(5)
        ]       
        
        self.depthwise_block7 = DepthwiseSeparableConvolution(conv_filters=(512,1024),conv_strides=(2,1),width_multiplier=self.width_multiplier)
        self.depthwise_block8 = DepthwiseSeparableConvolution(conv_filters=(1024,1024),conv_strides=(1,1),width_multiplier=self.width_multiplier)
        
        self.global_average_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.fully_connected = tf.keras.layers.Dense(units = 120)
        self.activation_fully_connected = tf.keras.layers.Activation('relu')
        
        
    def call(self, inputs, training=None):

        first_conv = self.conv1(inputs)
        first_conv = self.bn1(first_conv, training=training)
        first_conv = self.activation1(first_conv)
        
        dw_con1 = self.depthwise_block1(first_conv, training=training)
        dw_con2 = self.depthwise_block2(dw_con1, training=training)
        dw_con3 = self.depthwise_block3(dw_con2, training=training)
        dw_con4 = self.depthwise_block4(dw_con3, training=training)
        dw_con5 = self.depthwise_block5(dw_con4, training=training)
        dw_con6 = self.depthwise_block6(dw_con5, training=training)
        
        penta_block = dw_con6
        for dw in self.dw_penta_block:
            penta_block = dw(penta_block, training = training)
            
        
        dw_con7 = self.depthwise_block7(penta_block, training=training)
        dw_con8 = self.depthwise_block8(dw_con7, training=training)
        
        gap = self.global_average_pool(dw_con8)
        fc = self.fully_connected(gap)
        output = self.activation_fully_connected(fc)
        

        return output

In [25]:
class DepthwiseSeparableConvolution(tf.keras.models.Model):
    def __init__(self,conv_filters,conv_strides,width_multiplier, **kwargs):
        super(DepthwiseSeparableConvolution, self).__init__(**kwargs)
        
        self.dw_filter = conv_filters[0]
        self.pw_filter = conv_filters[1]
        
        self.dw_stride = conv_strides[0]
        self.pw_stride = conv_strides[1]
        
        self.width_multiplier = width_multiplier
    
    def build(self, input_shape):
        
        point_wise_filters = round(self.pw_filter * self.width_multiplier)
        
        self.depthwise_conv = tf.keras.layers.SeparableConv2D(
            filters=self.dw_filter,
            kernel_size=3,
            strides=self.dw_stride,
            padding='same',
            depth_multiplier=self.width_multiplier)
        self.bn_dw = tf.keras.layers.BatchNormalization()
        self.activation_dw = tf.keras.layers.Activation('relu')

        self.pointwise_conv = tf.keras.layers.Conv2D(
            filters=point_wise_filters,
            kernel_size=1,
            strides=self.pw_stride,
            padding='same')
        self.bn_pw = tf.keras.layers.BatchNormalization()
        self.activation_pw = tf.keras.layers.Activation('relu')
    
    def call(self, inputs, training=None):
        
        dw_conv = self.depthwise_conv(inputs)
        dw_conv = self.bn_dw(dw_conv, training=training)
        dw_conv = self.activation_dw(dw_conv)

        pw_conv = self.pointwise_conv(dw_conv)
        pw_conv = self.bn_pw(pw_conv, training=training)
        pw_conv = self.activation_pw(pw_conv)

        return pw_conv

## Model Summary

In [26]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
architecture = MobilNet_Architecture(width_multiplier=1)
x = architecture(inputs, training=False)
architecture.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_69 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization_132 (Bat (None, 112, 112, 32)      128       
_________________________________________________________________
activation_133 (Activation)  (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_separable_convolut (None, 112, 112, 64)      3840      
_________________________________________________________________
depthwise_separable_convolut (None, 56, 56, 128)       13824     
_________________________________________________________________
depthwise_separable_convolut (None, 56, 56, 128)       35200     
_________________________________________________________________
depthwise_separable_convolut (None, 28, 28, 256)       52224     
__________